In [25]:
import pandas as pd
import pyodbc 
from datetime import date,datetime, timedelta
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import MonthEnd
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
def leer_consulta(ruta_archivo):
    with open(ruta_archivo,'r',encoding='latin-1') as archivo:
        return archivo.read()
print('Librerias Cargadas.')
server = 'ECBPPRQ121,11124' 
database = 'DATALAB_IDN' 
username = 'mencarna' 
password = 'Pelus@141516' 
stringConn ='DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password+';Trusted_Connection=True;';
cnxn = pyodbc.connect(stringConn)
cursor = cnxn.cursor()
print('Conección Exitosa.')
Anio=2025
Mes=11
Tipo="Cierre"
L =[]
if Tipo=="Avance":
    fechas = datetime(Anio,Mes, datetime.now().day)- timedelta (days = 1)
    L=["BDDELIVERY.[IDN].[MasivoAhorro]","[DATALAB_MISMEDIOSPAGO].dbo.ZP_BP_Idn_TS_ProductividadMasivoCartera",
       "BDDELIVERY.[IDN].[MasivoPlazo]","[DATALAB_MISMEDIOSPAGO].dbo.ZP_BP_Idn_TS_ProductividadTarjetas"]
else:
    fechas = datetime(Anio,Mes,1)+ relativedelta(months=+1)- timedelta (days = 1)
    L=["DATALAB_IDN.[IDN].[MasivoAhorro1]","[DATALAB_MISMEDIOSPAGO].dbo.ZP_BP_Idn_TS_ProductividadMasivoCartera",
      "DATALAB_IDN.[IDN].[MasivoPlazo1]","[DATALAB_MISMEDIOSPAGO].dbo.ZP_BP_Idn_TS_ProductividadTarjetas"]
print('El proceso: '+str(Tipo)+',y Fecha ' + str(fechas) + '.'+L[1])  

Librerias Cargadas.
Conección Exitosa.
El proceso: Cierre,y Fecha 2025-11-30 00:00:00.[DATALAB_MISMEDIOSPAGO].dbo.ZP_BP_Idn_TS_ProductividadMasivoCartera


In [26]:
import os
# Ruta de los archivos SQL
ruta_fstp = '//ECUIO197H02/group_files/Inteligencia de Negocios/IDN-Global/Masivo/Querys/'
# Listar todos los archivos .sql en la ruta
archivos_sql = [archivo for archivo in os.listdir(ruta_fstp) if archivo.endswith('.sql')]
# Leer y concatenar las consultas usando "UNION ALL"
consulta_final = '\nUNION ALL\n'.join([f'{leer_consulta(ruta_fstp + archivo)}' for archivo in archivos_sql])
consulta_final = consulta_final.format(L = L, Anio = Anio, Mes = Mes)
df_Real = pd.read_sql(consulta_final, cnxn)
print('La obtención de Datos Reales de la Fuerza de Venta es Exitosa.')

C:\Users\mencarna\AppData\Local\Temp\ipykernel_23260\886435004.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_Real = pd.read_sql(consulta_final, cnxn)


La obtención de Datos Reales de la Fuerza de Venta es Exitosa.


# Metas

In [27]:
df = pd.read_excel('//ECUIO197H02/group_files/Inteligencia de Negocios/IDN-Global/Masivo/Masivo.xlsx',
                   header=[0,1,2],
                   index_col=[0],
                   sheet_name='Metas')
df = df.stack(level=1).stack(level=1).stack().unstack(level=1)
df = df.reset_index()
df=df.rename(columns = {'USUARIO':'PRODUCTO','level_0':'USUARIO','level_2':'PRODUCTOS'})
df = df[['PRODUCTO','USUARIO','METAS']]
df_BSC = pd.merge(df,df_Real,  how='left', 
             left_on=['PRODUCTO','USUARIO'], 
             right_on =['Producto','Usuario'] )
df_BSC =df_BSC[['PRODUCTO','USUARIO','Real','METAS']]
df = pd.read_excel('//ECUIO197H02/group_files/Inteligencia de Negocios/IDN-Global/Masivo/Masivo.xlsx',
                   sheet_name='Logica')
df_Final = pd.merge(df_BSC,df,  how='left', 
             left_on=['PRODUCTO'], 
             right_on =['PRODUCTO'] )
import pandas as pd
import datetime
df_Final['FECHA_CORTE'] = pd.to_datetime(fechas)
df_Final = df_Final.drop(['ANIO', 'MES'], axis=1)
df_Final["Real"].fillna(0, inplace=True)

C:\Users\mencarna\AppData\Local\Temp\ipykernel_23260\515503359.py:5: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).stack(level=1).stack().unstack(level=1)
C:\Users\mencarna\AppData\Local\Temp\ipykernel_23260\515503359.py:5: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=1).stack(level=1).stack().unstack(level=1)
C:\Users\mencarna\AppData\Local\Temp\ipykernel_23260\515503359.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will ch

In [28]:
query = f"""
select REGION,CodigoAgEjecucion,CodigoAgencia,Zona,AgenciaEjecucion,Agencia,Usuario,Asesor,Medibles,PorcMedicion,
ProvinciaAgencia,CiudadAgencia,CategorizacionAgencia,TipoMedicion 
from [DATALAB_MISNEGOCIOS].[HIS].[AsesoresAgencias]
where Anio={Anio} and Mes={Mes} and TipoAsesorProductividad='MULTIFUNCION' 
"""
df_HeatCount = pd.read_sql(query, cnxn)
df = pd.merge(df_Final,df_HeatCount,  how='left', 
             left_on=['USUARIO'], 
             right_on =['Usuario'] )
df = df.drop(['Usuario'], axis=1)
df = df[df['ACTIVO']=='SI']
df = df.fillna(0)

C:\Users\mencarna\AppData\Local\Temp\ipykernel_23260\1455758176.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_HeatCount = pd.read_sql(query, cnxn)
C:\Users\mencarna\AppData\Local\Temp\ipykernel_23260\1455758176.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)


In [29]:
cursor.execute("TRUNCATE TABLE [DATALAB_IDN].[IDN].[Banca_Multifuncion]")
for index, row in df.iterrows():
    cursor.execute("""INSERT INTO [DATALAB_IDN].[IDN].[Banca_Multifuncion] 
    (PRODUCTO,USUARIO,Real,METAS,CATEGORIA,ACTIVO,TECHO_PRODUCTO,TECHO_P,TECHO_C,PORCENTAJE,INVERTIBILIDAD,
    TIPO,VALOR,ERROR,FECHA_CORTE,REGION,CodigoAgEjecucion,CodigoAgencia,Zona,AgenciaEjecucion,Agencia,
    Asesor,Medibles,PorcMedicion,ProvinciaAgencia,CiudadAgencia,CategorizacionAgencia,TipoMedicion) 
    values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",
                   row.PRODUCTO,row.USUARIO,row.Real,row.METAS,row.CATEGORIA,row.ACTIVO,row.TECHO_PRODUCTO,
                   row.TECHO_P,row.TECHO_C,row.PORCENTAJE,row.INVERTIBILIDAD,row.TIPO,row.VALOR,row.ERROR,
                   row.FECHA_CORTE,row.REGION,row.CodigoAgEjecucion,row.CodigoAgencia,row.Zona,
                   row.AgenciaEjecucion,row.Agencia,row.Asesor,row.Medibles,row.PorcMedicion,
                   row.ProvinciaAgencia,row.CiudadAgencia,row.CategorizacionAgencia,row.TipoMedicion)
    cnxn.commit()
print('Completado')

Completado


In [30]:
import pandas as pd
import pyodbc 
from datetime import date,datetime, timedelta
from pandas.tseries.offsets import MonthEnd

server = 'ECBPPRQ121,11124' 
database = 'DATALAB_IDN' 
username = 'erisimba' 
password = 'Nomasclaves2024#' 
stringConn ='DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password+';Trusted_Connection=True;';
cnxn = pyodbc.connect(stringConn)
cursor = cnxn.cursor()
cursor.execute(f"""
	DELETE FROM [DATALAB_IDN].[HIS].[Banca_Multifuncion]
	WHERE YEAR(FECHA_CORTE) = {Anio} AND MONTH(FECHA_CORTE) = {Mes}
	INSERT INTO [DATALAB_IDN].[HIS].[Banca_Multifuncion]
	SELECT * FROM [DATALAB_IDN].[IDN].[Banca_Multifuncion]
    
    ---AJUSTE INS---
    UPDATE A
	SET TECHO_P=B.Piso,
		TECHO_C=B.Techo
	from
	[DATALAB_IDN].[HIS].[Banca_Multifuncion] a 
	inner JOIN (select codigoAgencia, CAST(REPLACE(piso, ',', '.') AS DECIMAL(10,2)) as Piso, CAST(REPLACE(Techo, ',', '.') AS DECIMAL(10,2)) as Techo  
	from [DATALAB_IDN].[dbo].[metasIndicadoresAgencias2] where anio={Anio} and mes={Mes} and tipo='INS' ) b on a.codigoAgEjecucion=b.codigoAgencia
	WHERE YEAR(a.FECHA_CORTE) = {Anio} AND MONTH(a.FECHA_CORTE) = {Mes} and producto='INS'
    
    ---AJUSTE SLA---
    UPDATE A
	SET A.METAS=B.Metas
	from
	[DATALAB_IDN].[HIS].[Banca_Multifuncion] a 
	inner JOIN (SELECT * FROM [DATALAB_IDN].[HIS].[Seguimiento_Agencias_Nivel_Uno] a
	where a.anio={Anio} and a.mes={Mes} and producto='NIVEL DE ATENCIÓN (SLA)') b on a.codigoAgEjecucion=b.CodigoAgenciaAnclaje
	WHERE YEAR(a.FECHA_CORTE) = {Anio} AND MONTH(a.FECHA_CORTE) = {Mes} and a.producto='SLA'
""")
cnxn.commit()
print('La Historia esta Cargada.')

La Historia esta Cargada.
